In [1]:
# author - Richard Liao 
# Dec 26 2016
import numpy as np
import pandas as pd

from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='tensorflow'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

#from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Lambda, Concatenate, Reshape
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session





Using TensorFlow backend.


In [2]:
def gpu_alloc(device_id):
    os.environ["CUDA_VISIBLE_DEVICES"]=device_id
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc("1")

# Data


In [4]:
MAX_WORD_LENGTH = 20
MAX_SENT_LENGTH = 40
MAX_SENTS = 10

VALIDATION_SPLIT = 0.2
NUM_EPOCHS = 100
BATCH_SIZE = 50

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

data_train = pd.read_csv('./dat/imdb/labeledTrainData.tsv', sep='\t')
print(data_train.shape)
data_train = data_train[:1000]
from nltk import tokenize

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    print('Parsing review ' + str(idx))
    text = BeautifulSoup(data_train.review[idx]).get_text()
    text = clean_str(text)#.get_text().encode('ascii','ignore'))
    #print('Text:\n' + text)
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    '''
    for sent in sentences:
          print('Sentence:\n' + sent)
    '''
    labels.append(data_train.sentiment[idx])





(25000, 3)
Parsing review 0
Parsing review 1
Parsing review 2
Parsing review 3
Parsing review 4
Parsing review 5
Parsing review 6
Parsing review 7
Parsing review 8
Parsing review 9
Parsing review 10
Parsing review 11
Parsing review 12
Parsing review 13
Parsing review 14
Parsing review 15
Parsing review 16
Parsing review 17
Parsing review 18
Parsing review 19
Parsing review 20
Parsing review 21
Parsing review 22
Parsing review 23
Parsing review 24
Parsing review 25
Parsing review 26
Parsing review 27
Parsing review 28
Parsing review 29
Parsing review 30
Parsing review 31
Parsing review 32
Parsing review 33
Parsing review 34
Parsing review 35
Parsing review 36
Parsing review 37
Parsing review 38
Parsing review 39
Parsing review 40
Parsing review 41
Parsing review 42
Parsing review 43
Parsing review 44
Parsing review 45
Parsing review 46
Parsing review 47
Parsing review 48
Parsing review 49
Parsing review 50
Parsing review 51
Parsing review 52
Parsing review 53
Parsing review 54
Parsing r

/opt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Parsing review 203
Parsing review 204
Parsing review 205
Parsing review 206
Parsing review 207
Parsing review 208
Parsing review 209
Parsing review 210
Parsing review 211
Parsing review 212
Parsing review 213
Parsing review 214
Parsing review 215
Parsing review 216
Parsing review 217
Parsing review 218
Parsing review 219
Parsing review 220
Parsing review 221
Parsing review 222
Parsing review 223
Parsing review 224
Parsing review 225
Parsing review 226
Parsing review 227
Parsing review 228
Parsing review 229
Parsing review 230
Parsing review 231
Parsing review 232
Parsing review 233
Parsing review 234
Parsing review 235
Parsing review 236
Parsing review 237
Parsing review 238
Parsing review 239
Parsing review 240
Parsing review 241
Parsing review 242
Parsing review 243
Parsing review 244
Parsing review 245
Parsing review 246
Parsing review 247
Parsing review 248
Parsing review 249
Parsing review 250
Parsing review 251
Parsing review 252
Parsing review 253
Parsing review 254
Parsing revi

Parsing review 807
Parsing review 808
Parsing review 809
Parsing review 810
Parsing review 811
Parsing review 812
Parsing review 813
Parsing review 814
Parsing review 815
Parsing review 816
Parsing review 817
Parsing review 818
Parsing review 819
Parsing review 820
Parsing review 821
Parsing review 822
Parsing review 823
Parsing review 824
Parsing review 825
Parsing review 826
Parsing review 827
Parsing review 828
Parsing review 829
Parsing review 830
Parsing review 831
Parsing review 832
Parsing review 833
Parsing review 834
Parsing review 835
Parsing review 836
Parsing review 837
Parsing review 838
Parsing review 839
Parsing review 840
Parsing review 841
Parsing review 842
Parsing review 843
Parsing review 844
Parsing review 845
Parsing review 846
Parsing review 847
Parsing review 848
Parsing review 849
Parsing review 850
Parsing review 851
Parsing review 852
Parsing review 853
Parsing review 854
Parsing review 855
Parsing review 856
Parsing review 857
Parsing review 858
Parsing revi

In [5]:
def build_chars_vocab(all_texts):
    '''Build vocab dictionary to victorize chars into ints'''
    vocab_to_int = {}
    count = 0 # Start index for any char will be 1, as 0 is masked by the Embedding/Masking layer
    
    for text in all_texts:
        for char in text:
            if char not in vocab_to_int:
                vocab_to_int[char] = count
                count += 1


    '''''Build inverse translation from int to char'''
    int_to_vocab = {}
    for character, value in vocab_to_int.items():
        int_to_vocab[value] = character
        
    return vocab_to_int, int_to_vocab

char2int, int2char = build_chars_vocab(texts)


In [6]:


data_lst = []
labels_lst = []
for i, sentences in enumerate(reviews):
    data = np.zeros((MAX_SENTS, MAX_SENT_LENGTH, MAX_WORD_LENGTH), dtype='int32')
    for _, sent in enumerate(sentences):
        j = 0
        if j<MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH:
                    l = 0
                    for char in word:
                         if l < MAX_WORD_LENGTH:
                                data[j,k,l] = char2int[char]
                                l=l+1
                    k=k+1
            j=j+1
    data_lst.append(data)
    labels_lst.append(labels[i])
data = np.array(data_lst)
labels = np.array(labels_lst)


                    
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Shape of data tensor: (1000, 15, 100, 40)
Shape of label tensor: (1000, 2)
Number of positive and negative reviews in traing and validation set
[ 413.  387.]
[ 105.   95.]


# Model


In [7]:
n_char_tokens = len(char2int)

char_emb_layer = Embedding(n_char_tokens, n_char_tokens,                           
                            weights=[np.eye(n_char_tokens)],
                            mask_zero=True, trainable=False)
'''
char_emb_layer = Embedding(n_char_tokens, 100,                           
                            mask_zero=True)
'''
word_enc_layer = Bidirectional(LSTM(100))

sent_enc_layer = Bidirectional(LSTM(100))

rev_enc_layer = Bidirectional(LSTM(100))

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH, MAX_WORD_LENGTH), dtype='int32')

s_embs = []
for i in range(MAX_SENTS):
    sent = Lambda(lambda x: x[:,i,:])(review_input)#sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    w_embs = []
    for j in range(MAX_SENT_LENGTH):
        word = Lambda(lambda x: x[:,j])(sent)
        '''
        c_embds = []
        for k in range(MAX_WORD_LENGTH):
            char = Lambda(lambda x: x[:,k])(word)
            c_emb = char_emb_layer(char)
            c_emb = Reshape((1,int(c_emb.shape[1])))(c_emb)
            c_embs.append(c_emb) 
            
        c_embs = Concatenate(axis=-2)(c_embs)    
        '''
        c_embs = char_emb_layer(word)
        w_emb = word_enc_layer(c_embs)
        w_emb = Reshape((1,int(w_emb.shape[1])))(w_emb)
        w_embs.append(w_emb)    
        
    w_embs = Concatenate(axis=-2)(w_embs)
    
    s_emb = sent_enc_layer(w_embs)
    s_emb = Reshape((1, int(s_emb.shape[1])))(s_emb)
    s_embs.append(s_emb)

s_embs = Concatenate(axis=-2)(s_embs)

rev_emb = rev_enc_layer(s_embs)

preds = Dense(2, activation='softmax')(rev_emb)
model = Model(review_input, preds)
'''
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)
print(sentEncoder.summary())
review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
print(review_encoder.shape)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)
'''
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 100, 40)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 100, 40)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_102 (Lambda)             (None, 100, 40)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_203 (Lambda)             (None, 100, 40)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_304

lambda_715 (Lambda)             (None, 40)           0           lambda_708[0][0]                 
__________________________________________________________________________________________________
lambda_716 (Lambda)             (None, 40)           0           lambda_708[0][0]                 
__________________________________________________________________________________________________
lambda_717 (Lambda)             (None, 40)           0           lambda_708[0][0]                 
__________________________________________________________________________________________________
lambda_718 (Lambda)             (None, 40)           0           lambda_708[0][0]                 
__________________________________________________________________________________________________
lambda_719 (Lambda)             (None, 40)           0           lambda_708[0][0]                 
__________________________________________________________________________________________________
lambda_720

lambda_1472 (Lambda)            (None, 40)           0           lambda_1415[0][0]                
__________________________________________________________________________________________________
lambda_1473 (Lambda)            (None, 40)           0           lambda_1415[0][0]                
__________________________________________________________________________________________________
lambda_1474 (Lambda)            (None, 40)           0           lambda_1415[0][0]                
__________________________________________________________________________________________________
lambda_1475 (Lambda)            (None, 40)           0           lambda_1415[0][0]                
__________________________________________________________________________________________________
lambda_1476 (Lambda)            (None, 40)           0           lambda_1415[0][0]                
__________________________________________________________________________________________________
lambda_147

bidirectional_1 (Bidirectional) (None, 200)          152000      embedding_1[0][0]                
                                                                 embedding_1[1][0]                
                                                                 embedding_1[2][0]                
                                                                 embedding_1[3][0]                
                                                                 embedding_1[4][0]                
                                                                 embedding_1[5][0]                
                                                                 embedding_1[6][0]                
                                                                 embedding_1[7][0]                
                                                                 embedding_1[8][0]                
                                                                 embedding_1[9][0]                
          

reshape_706 (Reshape)           (None, 1, 200)       0           bidirectional_1[699][0]          
__________________________________________________________________________________________________
reshape_708 (Reshape)           (None, 1, 200)       0           bidirectional_1[700][0]          
__________________________________________________________________________________________________
reshape_709 (Reshape)           (None, 1, 200)       0           bidirectional_1[701][0]          
__________________________________________________________________________________________________
reshape_710 (Reshape)           (None, 1, 200)       0           bidirectional_1[702][0]          
__________________________________________________________________________________________________
reshape_711 (Reshape)           (None, 1, 200)       0           bidirectional_1[703][0]          
__________________________________________________________________________________________________
reshape_71

                                                                 reshape_1398[0][0]               
                                                                 reshape_1399[0][0]               
                                                                 reshape_1400[0][0]               
                                                                 reshape_1401[0][0]               
                                                                 reshape_1402[0][0]               
                                                                 reshape_1403[0][0]               
                                                                 reshape_1404[0][0]               
                                                                 reshape_1405[0][0]               
                                                                 reshape_1406[0][0]               
                                                                 reshape_1407[0][0]               
          

In [ ]:


print("model fitting - H_LSTM char2doc")
NUM_EPOCHS = 2

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=NUM_EPOCHS, batch_size=BATCH_SIZE)


model fitting - H_LSTM char2doc


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 800 samples, validate on 200 samples
Epoch 1/2


# Test

In [ ]:

data_test = pd.read_csv('./dat/imdb/labeledTrainData.tsv', sep='\t')
print(data_test.shape)


reviews = []
texts = []

for idx in range(data_test.review.shape[0]):
    print('Parsing review ' + str(idx))
    text = BeautifulSoup(data_test.review[idx]).get_text()
    text = clean_str(text)#.get_text().encode('ascii','ignore'))

    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)




data_lst = []
labels_lst = []
for i, sentences in enumerate(reviews):
    data = np.zeros((MAX_SENTS, MAX_SENT_LENGTH,MAX_WORD_LENGTH), dtype='int32')
    for _, sent in enumerate(sentences):
        j = 0
        if j<MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH:
                    l = 0
                    for _, char in word:
                         if l < MAX_WORD_LENGTH:
                                data[j,k,l] = char2int[char]
                                l=l+1
                    k=k+1
            j=j+1
    data_lst.append(data)
    labels_lst.append(labels[i])
data = np.array(data_lst)

#labels = np.array(labels_lst)                 
                    
test_input_data = data
test_texts = list(data_test.review.apply(BeautifulSoup).apply(BeautifulSoup.get_text).apply(clean_str))

In [ ]:
for i, rev in enumerate(data_test.review):
    print(rev)
    test_input = test_input_data[i].copy()
    test_input = np.reshape(test_input, (1,test_input.shape[0], test_input.shape[1]))
    prediction = model.predict(test_input)
    print('Prediction: ', prediction)
    sentiment = np.argmax(prediction)
    print('Sentiment: ' + str(sentiment))